# Use mdapy efficiently

## The key point is re-using the neighborlist information.

In [1]:
import mdapy as mp
mp.init()

[Taichi] version 1.6.0, llvm 15.0.1, commit f1c6fbbd, win, python 3.10.0
[Taichi] Starting on arch=x64


In [2]:
mp.__version__

'0.9.4'

## Build a FCC lattice with 4,000,000 atoms.

In [3]:
FCC = mp.LatticeMaker(3.615, 'FCC', 100, 100, 100)
FCC.compute()

## Generate a system class.

In [4]:
system = mp.System(pos=FCC.pos, box=FCC.box, boundary=[1, 1, 1])

## Check the system information.

In [5]:
system

Filename: None
Atom Number: 4000000
Simulation Box:
[[361.5   0.    0. ]
 [  0.  361.5   0. ]
 [  0.    0.  361.5]
 [  0.    0.    0. ]]
TimeStep: 0
Boundary: [1, 1, 1]
Particle Information:
shape: (5, 5)
┌─────┬──────┬────────┬────────┬────────┐
│ id  ┆ type ┆ x      ┆ y      ┆ z      │
│ --- ┆ ---  ┆ ---    ┆ ---    ┆ ---    │
│ i32 ┆ i32  ┆ f64    ┆ f64    ┆ f64    │
╞═════╪══════╪════════╪════════╪════════╡
│ 1   ┆ 1    ┆ 0.0    ┆ 0.0    ┆ 0.0    │
│ 2   ┆ 1    ┆ 1.8075 ┆ 1.8075 ┆ 0.0    │
│ 3   ┆ 1    ┆ 1.8075 ┆ 0.0    ┆ 1.8075 │
│ 4   ┆ 1    ┆ 0.0    ┆ 1.8075 ┆ 1.8075 │
│ 5   ┆ 1    ┆ 0.0    ┆ 0.0    ┆ 3.615  │
└─────┴──────┴────────┴────────┴────────┘

If we want to do a series of analysis, we can re-use the neighbor to save time.

## Direct calculation.

In [6]:
%%time
system.cal_centro_symmetry_parameter(12)
system.cal_ackland_jones_analysis()
system.cal_atomic_entropy()

CPU times: total: 5min 51s
Wall time: 21.9 s


In [7]:
system.data.head()

id,type,x,y,z,csp,aja,atomic_entropy
i32,i32,f64,f64,f64,f64,i32,f64
1,1,0.0,0.0,0.0,3.2952e-26,1,-8.645167
2,1,1.8075,1.8075,0.0,1.0984e-26,1,-8.645167
3,1,1.8075,0.0,1.8075,1.0984e-26,1,-8.645167
4,1,0.0,1.8075,1.8075,1.0984e-26,1,-8.645167
5,1,0.0,0.0,3.615,2.1968e-26,1,-8.645167


## Re-use neighborlist information.

In [8]:
%%time
system.build_neighbor(rc=5.0, max_neigh=50) # Obtain the neighbor first, the following calculation can use it.
system.cal_atomic_entropy()
system.cal_ackland_jones_analysis()
system.cal_centro_symmetry_parameter(12)

CPU times: total: 4min 31s
Wall time: 13.4 s


In [9]:
system.data.head()

id,type,x,y,z,csp,aja,atomic_entropy
i32,i32,f64,f64,f64,f64,i32,f64
1,1,0.0,0.0,0.0,3.2952e-26,1,-8.645167
2,1,1.8075,1.8075,0.0,1.0984e-26,1,-8.645167
3,1,1.8075,0.0,1.8075,1.0984e-26,1,-8.645167
4,1,0.0,1.8075,1.8075,1.0984e-26,1,-8.645167
5,1,0.0,0.0,3.615,2.1968e-26,1,-8.645167
